In [1]:
#connecting workspace
from azureml.core import Workspace

ws = Workspace(subscription_id='738974c0-9684-498b-bbd0-c53d6d04d964',resource_group='rg-oof-cus-d-001',workspace_name='ML-OOF-CUS-D')

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.


In [2]:
#setting mlflow tracking
import mlflow
tracking_uri = ws.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(tracking_uri)

In [3]:
from urllib.parse import urlparse

urlparse(mlflow.get_tracking_uri()).scheme

'azureml'

In [4]:

mlflow.create_experiment('mlops_spam')
mlflow.set_experiment('mlops_spam')

<Experiment: artifact_location='', creation_time=1687734367400, experiment_id='97ed9d86-d892-4f0a-b115-bbbbceeba186', last_update_time=None, lifecycle_stage='active', name='mlops_spam', tags={}>

In [5]:
#directory 
import os
os.getcwd()

'c:\\Users\\AA706\\Desktop\\Mlops'

In [6]:
#loading binary dataset from  azure
from azureml.core.dataset import Dataset

cleaned_binary_data = Dataset.get_by_name(ws, 'cleaned_dataset')

cleaned_binary_data= cleaned_binary_data.to_pandas_dataframe()

In [7]:
cleaned_binary_data['preprocessedBody'][0]

'based vertically integrated jewellery distributor entered agreement go public via merger tmkr spacthe transaction implies pro forma enterprise value merged entity named listed ticker advised advisors'

In [8]:
#loading BERT model from sentencetransformer
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('all-mpnet-base-v2')
# bert_model = SentenceTransformer('bert-large-uncased')


In [9]:
 #encoding text using sentence transformer   

def encoding(model, df):



    print(f'shape of dataset going for training: {cleaned_binary_data.shape[0]}')

    list_proccessedBody = list(df['preprocessedBody'])
    tfidf_features = model.encode(list_proccessedBody, show_progress_bar=True)
    return tfidf_features

tfidf_features= encoding(bert_model,cleaned_binary_data)    
# print(f'shape of dataset going for training: {cleaned_binary_data.shape[0]}')

# list_proccessedBody = list(cleaned_binary_data['preprocessedBody'])
# tfidf_features = bert_model.encode(list_proccessedBody, show_progress_bar=True)


shape of dataset going for training: 141


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
#functions for training model for spam model
import lightgbm as lgbm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
from azureml.core.model import Model


def log_metrics(clf, X_test, y_test, run):
    preds = np.round(clf.predict(X_test))
    run.log("accuracy (test)", accuracy_score(y_test, preds))
    run.log("precision (test)", precision_score(y_test, preds))
    run.log("recall (test)", recall_score(y_test, preds))
    run.log("f1 (test)", f1_score(y_test, preds))

def train_model():
    X_train, X_test, y_train, y_test = train_test_split(tfidf_features, cleaned_binary_data['topic'], test_size=0.15, random_state=42,stratify=cleaned_binary_data ['topic'])
    train_data = lgbm.Dataset(data=X_train, label=y_train)
    test_data  = lgbm.Dataset(data=X_test, label=y_test)

    full_train= lgbm.Dataset(data=tfidf_features, label=cleaned_binary_data['topic'])



    mlflow.lightgbm.autolog()

    with mlflow.start_run() as my_run:
        lgbm_params = {
    'colsample_bytree':0.4955555555555555,
    'learning_rate':0.09473736842105263,
    'max_bin':140,
    'max_depth':6,
    'min_split_gain':0.10526315789473684,
    'n_estimators':25,
    'num_leaves':164,
    'reg_alpha':0.3157894736842105,
    'reg_lambda':0.3157894736842105
}
        evaluation_results = {}

        
        clf = lgbm.train(train_set=train_data,
                 params=lgbm_params,
                 valid_sets=[train_data, test_data], 
                 valid_names=['train', 'val'],
                 evals_result=evaluation_results,
                 num_boost_round=500
               
                )
        

        preds = np.round(clf.predict(X_test))
        acc= ( accuracy_score(y_test, preds))
        pr= (precision_score(y_test, preds))
        rec=(recall_score(y_test, preds))
        f1=(f1_score(y_test, preds))
        mlflow.log_metric('accuracy', acc)
        mlflow.log_metric('precison', pr)
        mlflow.log_metric('recall', rec)
        mlflow.log_metric('f1 score', f1)

        model_spam = lgbm.train(train_set=full_train,
                 params=lgbm_params,

                 num_boost_round=500
               
                )
        

        
        
        



    return my_run

In [11]:
run=train_model()

2023/07/22 10:59:29 WARNING mlflow.lightgbm: Failed to log dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
c:\Users\AA706\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\AA706\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31395
[LightGBM] [Info] Number of data points in the train set: 119, number of used features: 768
[LightGBM] [Info] Start training from score 0.663866
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

2023/07/22 10:59:50 WARNING mlflow.lightgbm: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/07/22 11:00:13 WARNING mlflow.lightgbm: Failed to log dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
c:\Users\AA706\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36285
[LightGBM] [Info] Number of data points in the train set: 141, number of used features: 768
[LightGBM] [Info] Start training from score 0.666667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

2023/07/22 11:00:17 ERROR mlflow.lightgbm: Failed to log feature importance plot. LightGBM autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "c:\Users\AA706\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\lightgbm.py", line 830, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "c:\Users\AA706\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\lightgbm.py", line 716, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "c:\Users\AA706\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\tracking\fluent.py", line 869, in log_artifact
    MlflowClient().log_artifact(run_id, local_path, artifact_path)
  File "c:\Users\AA706\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\tracking\client.py", line 1091, in log_artifact
    self._tracking_client.log_artifact(run_id, local_path, artifact_path)
  File "c:\Users\AA706\AppData\Lo

In [12]:
run_id = run.info.run_id  
run_id

'f2cdde2f-f3a1-4211-a467-1a98829c3cf4'

In [13]:
# model_uri = "runs:/8f5bb639-48f5-4033-bee5-e4c7a5165a49/model"  
model_uri = f"runs:/{run_id}/model"  

 # Replace <run_id> with the actual run ID  
registered_model = mlflow.register_model(model_uri=model_uri,name='mlops_spam'
                                                        
                                                )  
      


Registered model 'mlops_spam' already exists. Creating a new version of this model...
2023/07/22 11:00:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: mlops_spam, version 4
Created version '4' of model 'mlops_spam'.


In [14]:
# mlflow.end_run()
os.getcwd()

'c:\\Users\\AA706\\Desktop\\Mlops'

In [15]:
from mlflow.tracking import MlflowClient

run_result = MlflowClient().get_run(run.info.run_id)
print("params: {}".format(run_result.data.params))
print("metrics: {}".format(run_result.data.metrics))

params: {'colsample_bytree': '0.4955555555555555', 'learning_rate': '0.09473736842105263', 'max_bin': '140', 'max_depth': '6', 'min_split_gain': '0.10526315789473684', 'n_estimators': '25', 'num_leaves': '164', 'reg_alpha': '0.3157894736842105', 'reg_lambda': '0.3157894736842105', 'evals_result': '{}', 'num_boost_round': '500', 'feature_name': 'auto', 'categorical_feature': 'auto', 'early_stopping_rounds': 'None', 'verbose_eval': 'warn', 'keep_training_booster': 'False'}
metrics: {'train-l2': 0.046051126194527704, 'val-l2': 0.13232889119539004, 'accuracy': 0.8636363636363636, 'precison': 0.8333333333333334, 'recall': 1.0, 'f1 score': 0.9090909090909091}


In [16]:
artifacts = [f.path for f in MlflowClient().list_artifacts(run_result.info.run_id, "model")]
print("artifacts: {}".format(artifacts))

artifacts: ['model/MLmodel', 'model/conda.yaml', 'model/model.lgb', 'model/python_env.yaml', 'model/requirements.txt']


In [17]:
##Deployment

In [33]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = "heart-classifier-" + endpoint_suffix

print(f"Endpoint name: {endpoint_name}")

Endpoint name: heart-classifier-yzlmt


In [34]:
from mlflow.deployments import get_deploy_client

deployment_client = get_deploy_client(mlflow.get_tracking_uri())    

endpoint_config = {
    "auth_mode": "key",
    "identity": {
        "type": "system_assigned"
    }
}

endpoint_config_path = "endpoint_config.json"
with open(endpoint_config_path, "w") as outfile:
    outfile.write(json.dumps(endpoint_config))

endpoint = deployment_client.create_endpoint(
    name=endpoint_name,
    config={"endpoint-config-file": endpoint_config_path},
)


In [35]:
blue_deployment_name = "blue"
deploy_config = {
    "instance_type": "Standard_F4s_v2",
    "instance_count": 1,
}

deployment_config_path = "deployment_config.json"
with open(deployment_config_path, "w") as outfile:
    outfile.write(json.dumps(deploy_config))

In [36]:
blue_deployment = deployment_client.create_deployment(
    name=blue_deployment_name,
    endpoint=endpoint_name,
    model_uri=model_uri,
    config={"deploy-config-file": deployment_config_path},
)  

Registered model 'blue-model' already exists. Creating a new version of this model...
2023/07/21 14:45:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: blue-model, version 2
Created version '2' of model 'blue-model'.


..........................................................................................

In [39]:
testpoint =tfidf_features[138]

In [40]:
testpoint

array([ 5.60234413e-02,  2.84005757e-02,  1.03088813e-02, -2.27704924e-02,
        1.63359661e-02, -5.22418059e-02,  2.18415651e-02,  6.68313429e-02,
        3.46839637e-03, -3.52389812e-02, -5.36919907e-02,  7.99149834e-03,
       -2.29447596e-02,  9.49998572e-02, -3.88320200e-02,  1.01108842e-01,
        5.19427173e-02, -1.60811804e-02, -2.44169310e-02, -1.08934753e-02,
        3.08227981e-03,  2.71635000e-02,  4.10681441e-02, -9.48367349e-04,
        3.50946076e-02,  8.26063380e-03,  1.95533014e-03,  1.11406334e-02,
       -6.68296032e-03, -8.01497772e-02,  4.54217792e-02,  3.70200649e-02,
        2.11284799e-03,  2.75945049e-02,  1.82168503e-06, -6.97036386e-02,
       -2.02663261e-02, -1.77328347e-03, -4.19269167e-02,  8.01458489e-03,
        1.65219009e-02, -3.56095321e-02,  5.62371053e-02, -1.18173566e-03,
       -2.81768888e-02, -3.49375531e-02, -1.48292426e-02, -2.10070964e-02,
       -1.36332884e-02,  1.87534317e-02,  1.96958482e-02,  6.25888407e-02,
        5.32973856e-02,  

In [42]:
list(testpoint)

[0.05602344,
 0.028400576,
 0.010308881,
 -0.022770492,
 0.016335966,
 -0.052241806,
 0.021841565,
 0.06683134,
 0.0034683964,
 -0.03523898,
 -0.05369199,
 0.007991498,
 -0.02294476,
 0.09499986,
 -0.03883202,
 0.10110884,
 0.051942717,
 -0.01608118,
 -0.024416931,
 -0.010893475,
 0.0030822798,
 0.0271635,
 0.041068144,
 -0.00094836735,
 0.035094608,
 0.008260634,
 0.0019553301,
 0.011140633,
 -0.0066829603,
 -0.08014978,
 0.04542178,
 0.037020065,
 0.002112848,
 0.027594505,
 1.821685e-06,
 -0.06970364,
 -0.020266326,
 -0.0017732835,
 -0.041926917,
 0.008014585,
 0.0165219,
 -0.035609532,
 0.056237105,
 -0.0011817357,
 -0.028176889,
 -0.034937553,
 -0.014829243,
 -0.021007096,
 -0.013633288,
 0.018753432,
 0.019695848,
 0.06258884,
 0.053297386,
 0.002706946,
 0.017096631,
 0.00053086865,
 0.037836187,
 -0.025221525,
 0.061483487,
 0.0028578737,
 0.007194959,
 0.059075836,
 -0.07136282,
 0.014118535,
 -0.0030033,
 0.029202256,
 -0.023564693,
 -0.017058304,
 -0.040320028,
 0.003254183,

In [43]:
import numpy as np
import codecs, json 
testpoint =tfidf_features[118]
testpoint = testpoint.tolist() # nested lists with same data, indices


In [18]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
# data =  {
#   "input_data": 
#     [testpoint]

    
  
# }
data={
  "input_data": [
    [
    testpoint
    ]
  ]
}
body = str.encode(json.dumps(data))

url = 'https://heart-classifier-yzlmt.centralus.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'Fkvj6jWY8iQ5lxA98RH9dvt84RclGuFb' 
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'mlflow-spam-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)


    result = response.read()
    result_decode= result.decode('utf-8')
    float_value = float(list(map(float, result_decode.replace("[","").replace("]","").split()))[0])
    if (float_value)>0.5:
        print('The article is a good article--binary classified as 1')

    else:
        print('The article is a bad article--binary classified as 0')

    
    # print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


In [1]:
os.getcwd()

'c:\\Users\\AA706\\Desktop'

In [ ]:
from azureml.core.runconfig import RunConfiguration  
from azureml.core.conda_dependencies import CondaDependencies  
from azureml.core import Environment   
  
aml_run_config = RunConfiguration()  
aml_run_config.target = aml_cluster  
  
# Use a curated environment  
if use_curated_env:  
    curated_env = Environment.get(workspace=ws, name="AzureML-Tutorial").clone("mlops_env")  
    curated_env.python.conda_dependencies.add_conda_package("sentence-transformers")  
    curated_env.register(workspace=ws)  
    aml_run_config.environment = curated_env  
else:  
    aml_run_config.environment.python.user_managed_dependencies = True  
    dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'seaborn'],  
                                            pip_packages=['azureml-sdk', 'nltk', 'string', 're'],  
                                            channels=['conda-forge'])  
    aml_run_config.environment.python.conda_dependencies = dependencies  
